### Import bibliotek

In [1]:
import requests
from bs4 import BeautifulSoup

Wysłanie zapytania o dostęp do strony z opiniami o produkcie

In [ ]:
url = "https://www.ceneo.pl/84514582#tab=reviews"
response = requests.get(url)
response.status_code


Jeśli operacja zakończy się suckesem i otrzymamy kod strony, wyodrębniamy z kodu strony fragmenty odpowiadające poszczególnym opinionm.

In [3]:
if response.status_code == 200 :
    page_dom = BeautifulSoup(response.text, "html.parser")
    opinions = page_dom.select("div.js_product-review:not(.user-post--highlight)")
   

Dla każdej z opinii wydobywamy z kodu HTML poszczególne składowe i zapisanie ich w postaci struktur danych.

In [ ]:
for opinion in opinions:
   try:
      single_opinion = {
         "author":opinion.select_one("span.user-post__author-name").text.strip(),
         "recommendation":opinion.select_one("span.user-post__author-recomendation > em").text.strip(),
         "stars":opinion.select_one("span.user-post__score-count").text.strip(),
         "content":opinion.select_one("div.user-post__text").text.strip(),
         "likes":opinion.select_one("button.vote-yes > span").text.strip(),
         "dislikes":opinion.select_one("button.vote-no > span").text.strip(),
         "publish_date":opinion.select_one("span.user-post__published > time:nth-child(1)")['datetime'].strip(),
         "purchase_date":opinion.select_one("span.user-post__published >time:nth-child(2)")['datetime'].strip(),
         "pros":[p.text.strip() for p in opinion.select_one("div.review-feature__item--positive")],
         "cons":[c.text.strip() for c in opinion.select_one("div.review-feature__item--negative")],
         "review_id":opinion['data-entry-id']
      }  
      print(single_opinion)
   except (AttributeError, TypeError):
      pass  

